## Relational Data Model

### Defining a primary Key

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
dataPath_Raw = ("../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
dataPath_Processed = ("../data/processed/")

pd.set_option("display.max_rows", 500)

In [4]:
pd_raw = pd.read_csv(dataPath_Raw)
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,17267,18054,18969,19551,20342,20917,21459,22142,22890,23546
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1184,1197,1212,1232,1246,1263,1299,1341,1385,1416
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,9733,9831,9935,10050,10154,10265,10382,10484,10589,10698
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,851,852,852,852,852,852,852,852,852,853
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,86,86,86,88,91,92,96,113,118,130


In [5]:
pd_dataBase = pd_raw.rename(columns={'Country/Region':'country', 
                                     'Province/State':'state'})

In [6]:
pd_dataBase = pd_dataBase.drop(['Lat', 'Long'], axis=1)

In [7]:
pd_dataBase.head()

,state,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,17267,18054,18969,19551,20342,20917,21459,22142,22890,23546
1,NaN,Albania,0,0,0,0,0,0,0,0,...,1184,1197,1212,1232,1246,1263,1299,1341,1385,1416
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,9733,9831,9935,10050,10154,10265,10382,10484,10589,10698
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,851,852,852,852,852,852,852,852,852,853
4,NaN,Angola,0,0,0,0,0,0,0,0,...,86,86,86,88,91,92,96,113,118,130


In [8]:
pd_dataBase.set_index(['state', 'country']);

In [9]:
test_pd = pd_dataBase.set_index(['state', 'country']).T

In [10]:
test_pd.columns;

In [11]:
test_pd.stack(level=[0,1]).reset_index();

In [12]:
# As a sequence

pd_relational_model = pd_dataBase.set_index(['state', 'country']) \
                        .T \
                        .stack(level=[0,1]) \
                        .reset_index() \
                        .rename(columns={'level_0' : 'date',
                                        0 : 'confirmed'})

pd_relational_model.head()  


,date,state,country,confirmed
0,1/22/20,NaN,Afghanistan,0.0
1,1/22/20,NaN,Albania,0.0
2,1/22/20,NaN,Algeria,0.0
3,1/22/20,NaN,Andorra,0.0
4,1/22/20,NaN,Angola,0.0


In [13]:
pd_relational_model.dtypes

date          object
state         object
country       object
confirmed    float64
dtype: object

In [14]:
pd_relational_model.to_csv(dataPath_Processed + 'COVID_relational_Confirmed.csv', sep=',', index=False)

### Group-by apply

In [16]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=',',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


### Test Data

In [17]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [19]:
test_data.head()

,date,state,country,confirmed
15736,2020-03-21,no,Germany,22213.0
15776,2020-03-21,no,US,25733.0
16002,2020-03-22,no,Germany,24873.0
16042,2020-03-22,no,US,33660.0
16269,2020-03-23,no,Germany,29056.0


In [20]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-06-12,no,187226.0
US,2020-06-12,no,2048986.0


In [26]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope



if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

   


the test slope is: [2.]


In [27]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-06-12   187226.0
      US      2020-06-12  2048986.0

In [28]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [29]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15736            NaN
                16002            NaN
                16269       7.417994
                16534       7.142035
                16800       8.012983
                17067       6.954407
                17333       6.501919
                17598       7.390371
                17864      10.136671
                18130      13.541893
                18396      13.781393
                18663      13.140681
                18928      12.037271
                19193      12.735506
                19461      16.052694
                19726      21.372453
                19992      27.427355
                20259      27.511936
                20524      21.792179
                20790      21.495848
                21057      26.565108
                21322      36.198404
                21587      43.982990
                21855      49.423444
                22121      74.043747
                22386      56.424411
                

In [30]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [42]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,732,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1232,NaN


In [32]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [33]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


## Filtering the data with groupby apply 


In [34]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [35]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [36]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


## Filtered doubling rate

In [37]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
38033,no,Zimbabwe,36772,21.510101
38034,no,Zimbabwe,37069,27.432099
38035,no,Zimbabwe,37304,26.976608
38036,no,Zimbabwe,37570,25.354497
38037,no,Zimbabwe,38037,25.553846


In [38]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
38033,38033,2020-06-12,no,Barbados,96.0,inf,96.8,79.666667
38034,38034,2020-06-12,no,Belarus,52520.0,71.252636,52561.0,67.394275
38035,38035,2020-06-12,no,Belgium,59819.0,477.597333,59820.0,490.940789
38036,38036,2020-06-12,no,Albania,1416.0,36.817778,1419.2,35.204082
38037,38037,2020-06-12,no,Zimbabwe,343.0,28.840580,345.2,25.553846


In [39]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [40]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
36750,36750,2020-06-08,no,Germany,186109.0,563.792615,186067.4,511.340125
37016,37016,2020-06-09,no,Germany,186506.0,492.385362,186315.6,655.313843
37282,37282,2020-06-10,no,Germany,186522.0,902.561743,186610.8,685.797816
37548,37548,2020-06-11,no,Germany,186691.0,2017.005405,186852.7,694.816732
37815,37815,2020-06-12,no,Germany,187226.0,530.718750,187094.6,772.437784


In [41]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=',',index=False)